<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Notion - Update database with GitHub repositories info

**Tags:** #notion #github #database #update #repositories #python

**Author:** [Florent Ravenel](http://linkedin.com/in/florent-ravenel)

**Description:** This notebook will update Notion database with repositories info from your GitHub organization.

**References:**
- [Notion API Documentation](https://notion-py.readthedocs.io/en/latest/)
- [GitHub API Documentation](https://developer.github.com/v3/)

## Input

### Import libraries

In [ ]:
import notion
import github

### Setup Variables
- **notion_token**: [Generate Notion token](https://www.notion.so/api/v3/generate-token)
- **github_token**: [Generate GitHub token](https://github.com/settings/tokens)
- **org_name**: Name of the GitHub organization

In [ ]:
notion_token = "<notion_token>"
github_token = "<github_token>"
org_name = "<org_name>"

## Model

### Get repositories info

Long description of the function: This function will get the list of repositories from the GitHub organization and store the info in a list.

In [ ]:
# Create Notion and GitHub clients
notion_client = notion.NotionClient(token_v2=notion_token)
github_client = github.Github(github_token)
# Get the list of repositories from the GitHub organization
org = github_client.get_organization(org_name)
repos = org.get_repos()
# Store the info in a list
repos_info = []
for repo in repos:
    repos_info.append(
        {"name": repo.name, "url": repo.html_url, "description": repo.description}
    )

### Update Notion database

Long description of the function: This function will update the Notion database with the repositories info.

In [ ]:
# Get the Notion database
db_url = "<database_url>"
db = notion_client.get_block(db_url)
# Update the Notion database
for repo in repos_info:
    row = db.collection.add_row()
    row.name = repo["name"]
    row.url = repo["url"]
    row.description = repo["description"]

## Output

### Display result

In [ ]:
# Display the list of repositories
for repo in repos_info:
    print(f"Name: {repo['name']}")
    print(f"URL: {repo['url']}")
    print(f"Description: {repo['description']}")
    print()